### 🎯 __Project Objectives: A/B Test — “recommender_system_test”__

The main objective of this second phase of the data analysis project is to evaluate the impact of implementing a new recommendation system on the behavior of users of an international online store, through the analysis of a controlled A/B test.

Specifically, the objectives are:

1. Measure and compare the performance of the new checkout funnel (group B) against the current funnel (group A), using key conversion metrics within the user journey:

    - Product page views (product_page)
    - Add to cart events (product_cart)
    - Completed purchases (purchase)

2. Validate whether the new recommendation system improves sales funnel conversion by at least 10% at each stage, within 14 days of user registration, as established in the business hypothesis.

3. Verify the consistency and quality of the collected data to ensure the analysis is statistically reliable and representative of the selected audience (15% of new users in the EU region).

4. Apply appropriate statistical methods (such as hypothesis testing and significance analysis) to determine whether the observed differences between groups A and B are attributable to the new recommendation system and not due to chance.

5. Communicate actionable findings that allow the product team to decide whether the new checkout funnel should be implemented widely or requires additional adjustments before full deployment.

#### __Technical Description__

- Test Name: __recommender_system_test__
- Groups: A (control), B (new checkout funnel)
- Launch Date: 2020-12-07
- Date new users stopped: 2020-12-21
- End Date: 2021-01-01
- Audience: __15% of new__ users from the _EU region_
- Test Purpose: To test changes related to the introduction of an improved recommendation system
- Expected Outcome: __Within 14 days of enrollment__, users will show improved conversion rates for product page views (the product_page event), adding items to the shopping cart (product_cart), and purchases (purchase). At each stage of the `product_page → product_cart → purchase` __funnel__, there will be _at least a 10% increase_.
- Expected number of test participants: 6,000


####  __Data Description__

`ab_project_marketing_events_us.csv` (the marketing events calendar for 2020)

- `name`: the name of the marketing event
- `regions`: regions where the advertising campaign will be run
- `start_dt`: the campaign start date
- `finish_dt`: the campaign end date

`final_ab_new_users_upd_us.csv` (all users who registered in the online store from December 7 to 21, 2020)

- `user_id`: user identification
- `first_date`: Date of enrollment
- `region`: location
- `device`: Device used for enrollment

`final_ab_events_upd_us.csv` (all new user events from December 7, 2020, to January 1, 2021)

- `user_id`: user identification
- `event_dt`: Date and time of the event
- `event_name`: Name of the event type
- `details`: Additional information about the event (e.g., the total order in USD for purchase events)

`final_ab_participants_upd_us.csv` (a table with data on test participants)

- `user_id`: user identification
- `ab_test`: Name of the test
- `group`: The test group the user belonged to

### 💻 __1. Notebook Libraries and Customization__

In [1]:
import datetime as dt
from datetime import datetime
from IPython.display import display, HTML
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
from scipy import stats as st
from statsmodels.stats.proportion import proportions_ztest
import unicodedata

### 💻 __2. Functions__

In [2]:
# Function to normalize string formatting in object-type columns
def normalize_string_format(df, include=None, exclude=None):
    """
    Standardizes text formatting for object-type (string) columns in a DataFrame.

    Operations performed:
    - Converts text to lowercase
    - Strips leading/trailing whitespace
    - Replaces punctuation with spaces
    - Collapses spaces into underscores
    - Removes redundant underscores
    - Adds unicode normalization to remove accents and special characters.

    Parameters:
    df (DataFrame): The input DataFrame.
    include (list, optional): Specific columns to apply formatting to. If None, applies to all except those in 'exclude'.
    exclude (list, optional): Columns to skip.

    Returns:
    DataFrame: Updated DataFrame with normalized string formats.
    """

    if exclude is None:
        exclude = []

    if include is None:
        available_columns = [col for col in df.columns if col not in exclude]
    else:
        available_columns = [col for col in include if col not in exclude]

    def clean_text(text):
        if isinstance(text, str):
            text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
            text = (text
                    # .lower()
                    .strip())
            text = re.sub(r'[^\w\s.]', ' ', text)    # Replaces all characters other than letters, numbers, or spaces with a space
            text = re.sub(r'\s+', '_', text)        # Replace one or more consecutive spaces with an underscore "_"
            text = re.sub(r'__+', '_', text)        # Replaces multiple consecutive underscores with a single "_"
            text = re.sub(r'_(?=\s|$)', '', text)   # Removes underscores that are just before a space or at the end of the text
            text = re.sub(r'__+', '_', text)        # Replaces double underscores with single underscores
        return text

    for column in available_columns:
        if df[column].dtype in ['object', 'string']:
            df[column] = df[column].apply(clean_text)

    return df

# Function to identify non-standard missing values in object-type columns
def check_existing_missing_values(df):
    """
    Checks object-type columns in a DataFrame for non-standard missing values.

    Parameters:
    df (DataFrame): The dataset to inspect.

    Output:
    Displays the number of non-standard missing entries per column and the matched values.
    """

    # Common non-standard representations of missing values
    missing_values = ['', ' ', 'N/A', 'none', 'None','null', 'NULL', 'NaN', 'nan', 'NAN', 'nat', 'NaT']

    display(HTML(f"<h4>Scanning for Non-Standard Missing Values</h4>"))

    for column in df.columns:

        matches = df[df[column].isin(missing_values)][column].unique()

        if df[column].isin(missing_values).any() and matches.size > 0:
            count = df[column].isin(missing_values).sum()
            display(
                HTML(f"> Missing values in column <i>'{column}'</i>: <b>{count}</b>"))
            display(
                HTML(f"&emsp;Matched non-standard values: {list(matches)}"))
        else:
            display(
                HTML(f"> Missing values in column <i>'{column}'</i>: None"))

    print()

    return None

# Function to standardize non-standard missing values to pd.NA
def replace_missing_values(df, include=None, exclude=None):
    """
    Replaces common non-standard missing value entries in object-type columns with pd.NA.

    Parameters:
    df (DataFrame): The input dataset.
    include (list, optional): List of columns to include. If None, all columns except those in 'exclude' are considered.
    exclude (list, optional): List of columns to exclude from replacement.

    Returns:
    DataFrame: Updated DataFrame with non-standard missing values replaced by pd.NA.
    """

    missing_values = ['', ' ', 'N/A', 'none', 'None', 'null', 'NULL', 'NaN', 'nan', 'NAN', 'nat', 'NaT']

    if exclude is None:
        exclude = []

    if include is None:
        available_columns = [col for col in df.columns if col not in exclude]
    else:
        available_columns = [col for col in include if col not in exclude]

    for column in available_columns:
        if df[column].dtype in ['object', 'string'] and df[column].isin(missing_values).any():
            df.loc[:, column] = df[column].replace(missing_values, pd.NA)

    return df

# function for displaying the percentage of mising values in a Dataset
def missing_values_rate(df, include=None, exclude=None):
    
    """
    Displays the percentage of missing values for specified columns in a DataFrame.

    Parameters:
    ----------
    df : pandas.DataFrame
        The DataFrame to analyze.

    include : list, optional
        List of column names to include in the analysis. If None, all columns not in `exclude` are considered.

    exclude : list, optional
        List of column names to exclude from the analysis. Default is an empty list.

    Returns:
    -------
    None
        Displays HTML output in a Jupyter Notebook environment.
    """
    
    if exclude is None:
        exclude = []

    if include is None:
        available_columns = [col for col in df.columns if col not in exclude]
    else:
        available_columns = [col for col in include if col not in exclude]

    for column in available_columns:
        total_values = len(df[column])
        if total_values == 0:
            percentage = 0
        else:
            missing_values = df[column].isna().sum()
            percentage = (missing_values / total_values) * 100

        display(HTML(f"> Percentage of missing values for column <i>'{column}'</i>: <b>{percentage:.2f}</b> %<br>"))
        display(HTML(f">    Total values: {df[column].shape[0]}<br>   > Missing values: {df[column].isna().sum()}<br><br>"))

# Funnel detect sequence
def detect_irregular_funnel_sequences(df_events, user_col='user_id', event_col='event_name', date_col='event_dt', time_col='event_tm'):
    """
    Detects users whose sequence of events by date does not follow the logical order of the funnel,
    allowing incomplete but ordered sequences.

    Returns:
    - df_users_irregular: DataFrame with columns:
    - user_id
    - event_date: Date of the event group
    - event_sequence: Ordered list of tuples (timestamp, event)
    """
    
    expected_order = ['login', 'product_page', 'product_cart', 'purchase']
    df_events = df_events.copy()

    # Combine date and time into a complete datetime 
    df_events['event_timestamp'] = df_events.apply(lambda row: datetime.combine(row[date_col], row[time_col]), axis=1) 

    # Sort by user, date and time 
    df_sorted = df_events.sort_values(by=[user_col, date_col, 'event_timestamp']) 

    irregular_records = [] 

    # Group by user and date 
    for (user_id, event_date), group in df_sorted.groupby([user_col, date_col]):
        funnel_events = group[group[event_col].isin(expected_order)] 

        # Generate list of tuples (timestamp, event) 
        event_sequence = list(zip(funnel_events['event_timestamp'].tolist(), funnel_events[event_col].tolist())) 

        # Check if the sequence is ordered according to the funnel 
        last_index = -1 
        for _, e in event_sequence:
            current_index = expected_order.index(e) 
            if current_index < last_index:
                irregular_records.append({'user_id': user_id, 'event_date': event_date, 'event_sequence': event_sequence}) 
                break 
            last_index = current_index 

    df_users_irregular = pd.DataFrame(irregular_records) 
    return df_users_irregular

# Plot pie or donut chart
# plot_pie_chart(df_users_testAB, 'region', 'user_id', color_map=color_map, unique_values=True)
def plot_pie_chart_plotlypx(df, category_col, value_col, color_map=None, labels=True, title=None, unique_values=False):
    """
    Generates a pie chart using Plotly Express.

    Parameters:

    - df: Input DataFrame
    - category_col: Column with categories (pie labels)
    - value_col: Column with values (proportions)
    - color_map: dictionary (color map by category) or list (sequence of colors)
    - labels: bool, if True shows labels as percentages
    - title: chart title text (None uses generic title)
    """
    
    if unique_values:
        df_plot = (
            df[[category_col, value_col]]
            .drop_duplicates()
            .groupby(category_col, observed=True)[value_col]
            .nunique()
            .reset_index(name='count')
        )
    else:
        df_plot = (
            df[[category_col, value_col]]
            .groupby(category_col, observed=True)[value_col]
            .count()
            .reset_index(name='count')
        )   
    
    # Create chart based on color_map type
    if isinstance(color_map, dict):
        fig = px.pie(
            df_plot,
            names=category_col,
            values='count',
            color=category_col,
            color_discrete_map=color_map,
            hole=0.0 # for full pie (adjustable if you want a donut-like pie)
        )
    elif isinstance(color_map, list):
        fig = px.pie(
            df_plot,
            names=category_col,
            values='count',
            color=category_col,
            color_discrete_sequence=color_map,
            hole=0.0 # for full pie (adjustable if you want a donut-like pie)
        )
    else:
        fig = px.pie(
            df_plot,
            names=category_col,
            values='count',
            color=category_col,
            hole=0.0 # for full pie (adjustable if you want a donut-like pie)
        )

    fig.update_traces(textinfo='percent+label' if labels else 'none')
    fig.update_layout(title_text=title if title else "Distribution by Category", title_x=0.5)
    fig.show()

# Function to plot a horizontal bar chart using Plotly Express
def plot_horizontal_bar_plotpx(df, x: str, y: str, hue: str | None = None,         # categorical column for grouping (like hue in seaborn)
                               title: str = '', xlabel: str = '', ylabel: str = '', sort: bool = True, height: int = 500, width: int = 1200, color: str = 'grey'):
    """
    Plots a horizontal bar chart with Plotly Express.

    Parameters:
    - df (DataFrame): Input DataFrame.
    - x (str): Column name for x-axis values (numeric).
    - y (str): Column name for y-axis categories.
    - hue (str, optional): Column for coloring/grouping (categorical).
    - title (str): Chart title.
    - xlabel (str): X-axis label.
    - ylabel (str): Y-axis label.
    - sort (bool): If True, sorts bars by x descending.
    - height (int): Figure height.
    - width (int): Figure width.
    """

    data = df.copy()
    if sort:
        data = data.sort_values(by=x, ascending=True)
        
    if hue:
        fig = px.bar(
            data,
            x=x,
            y=y,
            orientation='h',
            color=hue,             # uses categoric columns
            title=title,
            height=height,
            width=width
        )
    else:
        fig = px.bar(
            data,
            x=x,
            y=y,
            title=title,
            height=height,
            width=width,
            color_discrete_sequence=[color]
        )

    fig.update_layout(
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        template='plotly_white'
    )

    fig.show()


### 🔁 __3. Data Loading__

In [3]:
df_mkt_events = pd.read_csv('../data/raw/ab_project_marketing_events_us.csv', sep=',', header='infer', keep_default_na=False)
df_user_events = pd.read_csv('../data/raw/final_ab_events_upd_us.csv', sep=',', header='infer', keep_default_na=False)
df_users_registration = pd.read_csv('../data/raw/final_ab_new_users_upd_us.csv', sep=',', header='infer', keep_default_na=False)
df_user_test = pd.read_csv('../data/raw/final_ab_participants_upd_us.csv', sep=',', header='infer', keep_default_na=False)

### 🧹 __4. Data Cleanup__

##### **4.1** Data Overview

In [4]:
df_mkt_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [5]:
df_user_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     423761 non-null  object
 1   event_dt    423761 non-null  object
 2   event_name  423761 non-null  object
 3   details     423761 non-null  object
dtypes: object(4)
memory usage: 12.9+ MB


In [6]:
df_users_registration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [7]:
df_user_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


##### **4.2** Standardization of column heading formats (Lower case, snake case)

In [8]:
df_mkt_events = normalize_string_format(df_mkt_events, exclude=['start_dt', 'finish_dt'])
df_mkt_events

,name,regions,start_dt,finish_dt
0,Christmas_New_Year_Promo,EU_N.America,2020-12-25,2021-01-03
1,St._Valentine_s_Day_Giveaway,EU_CIS_APAC_N.America,2020-02-14,2020-02-16
2,St._Patric_s_Day_Promo,EU_N.America,2020-03-17,2020-03-19
3,Easter_Promo,EU_CIS_APAC_N.America,2020-04-12,2020-04-19
4,4th_of_July_Promo,N.America,2020-07-04,2020-07-11
5,Black_Friday_Ads_Campaign,EU_CIS_APAC_N.America,2020-11-26,2020-12-01
6,Chinese_New_Year_Promo,APAC,2020-01-25,2020-02-07
7,Labor_day_May_1st_Ads_Campaign,EU_CIS_APAC,2020-05-01,2020-05-03
8,International_Women_s_Day_Promo,EU_CIS_APAC,2020-03-08,2020-03-10
9,Victory_Day_CIS_May_9th_Event,CIS,2020-05-09,2020-05-11


In [9]:
df_users_registration = normalize_string_format(df_users_registration, include=['region', 'device'])
df_users_registration

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


##### **4.3** Explicit Duplicate Removal

In [10]:
display(HTML(f"> Explicit duplicates in <i>df_mkt_events</i>: <b>{df_mkt_events.duplicated().sum()}</b>"))
print()
display(HTML(f"> Explicit duplicates in <i>df_users_registration</i>: <b>{df_users_registration.duplicated().sum()}</b>"))
print()
display(HTML(f"> Explicit duplicates in <i>df_user_events</i>: <b>{df_user_events.duplicated().sum()}</b>"))
print()
display(HTML(f"> Explicit duplicates in <i>df_user_test</i>: <b>{df_user_test.duplicated().sum()}</b>"))

In [11]:
print(df_user_test.head())

            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test


In [12]:
print(df_user_test.tail())

                user_id group            ab_test
14520  1D302F8688B91781     B  interface_eu_test
14521  3DE51B726983B657     A  interface_eu_test
14522  F501F79D332BE86C     A  interface_eu_test
14523  63FBE257B05F2245     A  interface_eu_test
14524  79F9ABFB029CF724     B  interface_eu_test


##### **4.4** Missing Value Analysis

In [13]:
check_existing_missing_values(df_mkt_events)

In [14]:
check_existing_missing_values(df_users_registration)

In [15]:
check_existing_missing_values(df_user_events)

In [16]:
check_existing_missing_values(df_user_test)

In [17]:
# Handling missing values
df_user_events = replace_missing_values(df_user_events, include=['details'])
df_user_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,<NA>
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,<NA>
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,<NA>
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,<NA>


In [18]:
missing_values_rate(df_user_events, include=['details'])

`LSPL`
__Note:__

Missing values will not be imputated with any values, due missing values belong to other events than purchase and makes no sense to impute them with arbitrary values, because this would introduce noise or false data into the analysis.

- Do not impute: Correct, NaNs indicate that the information is not applicable.
- Safe quantitative operations: Correct, Pandas ignores NaNs in calculations.
- This keeps your analysis clean and consistent for each stage of the funnel.

##### **4.5** Casting Datatypes

In [19]:
# Cast to datetime
df_mkt_events['start_dt'] = pd.to_datetime(df_mkt_events['start_dt'], errors='coerce').dt.date
df_mkt_events['finish_dt'] = pd.to_datetime(df_mkt_events['finish_dt'], errors='coerce').dt.date
df_mkt_events

,name,regions,start_dt,finish_dt
0,Christmas_New_Year_Promo,EU_N.America,2020-12-25,2021-01-03
1,St._Valentine_s_Day_Giveaway,EU_CIS_APAC_N.America,2020-02-14,2020-02-16
2,St._Patric_s_Day_Promo,EU_N.America,2020-03-17,2020-03-19
3,Easter_Promo,EU_CIS_APAC_N.America,2020-04-12,2020-04-19
4,4th_of_July_Promo,N.America,2020-07-04,2020-07-11
5,Black_Friday_Ads_Campaign,EU_CIS_APAC_N.America,2020-11-26,2020-12-01
6,Chinese_New_Year_Promo,APAC,2020-01-25,2020-02-07
7,Labor_day_May_1st_Ads_Campaign,EU_CIS_APAC,2020-05-01,2020-05-03
8,International_Women_s_Day_Promo,EU_CIS_APAC,2020-03-08,2020-03-10
9,Victory_Day_CIS_May_9th_Event,CIS,2020-05-09,2020-05-11


In [20]:
df_users_registration['first_date'] = pd.to_datetime(df_users_registration['first_date'], errors='coerce').dt.date
df_users_registration

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


In [21]:
df_user_events['event_dt'] = pd.to_datetime(df_user_events['event_dt'], errors='coerce')
df_user_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,<NA>
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,<NA>
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,<NA>
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,<NA>


In [22]:
# Cast to category
df_mkt_events['regions'] = df_mkt_events['regions'].astype('category')
df_mkt_events

,name,regions,start_dt,finish_dt
0,Christmas_New_Year_Promo,EU_N.America,2020-12-25,2021-01-03
1,St._Valentine_s_Day_Giveaway,EU_CIS_APAC_N.America,2020-02-14,2020-02-16
2,St._Patric_s_Day_Promo,EU_N.America,2020-03-17,2020-03-19
3,Easter_Promo,EU_CIS_APAC_N.America,2020-04-12,2020-04-19
4,4th_of_July_Promo,N.America,2020-07-04,2020-07-11
5,Black_Friday_Ads_Campaign,EU_CIS_APAC_N.America,2020-11-26,2020-12-01
6,Chinese_New_Year_Promo,APAC,2020-01-25,2020-02-07
7,Labor_day_May_1st_Ads_Campaign,EU_CIS_APAC,2020-05-01,2020-05-03
8,International_Women_s_Day_Promo,EU_CIS_APAC,2020-03-08,2020-03-10
9,Victory_Day_CIS_May_9th_Event,CIS,2020-05-09,2020-05-11


In [23]:
df_users_registration['region'] = df_users_registration['region'].astype('category')
df_users_registration['device'] = df_users_registration['device'].astype('category')
df_users_registration

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


In [24]:
df_user_events['event_name'] = df_user_events['event_name'].astype('category')
df_user_events

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,<NA>
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,<NA>
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,<NA>
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,<NA>


In [25]:
df_user_test['group'] = df_user_test['group'].astype('category')
df_user_test['ab_test'] = df_user_test['ab_test'].astype('category')
df_user_test

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


In [26]:
# Check dtypes
df_mkt_events.dtypes

name           object
regions      category
start_dt       object
finish_dt      object
dtype: object

In [27]:
df_users_registration.dtypes

user_id         object
first_date      object
region        category
device        category
dtype: object

In [28]:
df_user_events.dtypes

user_id               object
event_dt      datetime64[ns]
event_name          category
details               object
dtype: object

In [29]:
df_user_test.dtypes

user_id      object
group      category
ab_test    category
dtype: object

### ⚙️ **5. Feature Engineering**

In [30]:
df_users = df_user_events.merge(df_users_registration, on='user_id', how='left')
df_users

,user_id,event_dt,event_name,details,first_date,region,device
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99,2020-12-07,N.America,iPhone
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99,2020-12-07,EU,iPhone
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99,2020-12-07,N.America,iPhone
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99,2020-12-07,EU,iPhone
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99,2020-12-07,EU,Android
...,...,...,...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,<NA>,2020-12-07,EU,Android
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,<NA>,2020-12-07,EU,PC
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,<NA>,2020-12-07,EU,Android
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,<NA>,2020-12-07,EU,iPhone


In [31]:
df_users['event_tm'] = df_users['event_dt'].dt.time
df_users['event_dt'] = df_users['event_dt'].dt.date
df_users

,user_id,event_dt,event_name,details,first_date,region,device,event_tm
0,E1BDDCE0DAFA2679,2020-12-07,purchase,99.99,2020-12-07,N.America,iPhone,20:22:03
1,7B6452F081F49504,2020-12-07,purchase,9.99,2020-12-07,EU,iPhone,09:22:53
2,9CD9F34546DF254C,2020-12-07,purchase,4.99,2020-12-07,N.America,iPhone,12:59:29
3,96F27A054B191457,2020-12-07,purchase,4.99,2020-12-07,EU,iPhone,04:02:40
4,1FD7660FDF94CA1F,2020-12-07,purchase,4.99,2020-12-07,EU,Android,10:15:09
...,...,...,...,...,...,...,...,...
423756,245E85F65C358E08,2020-12-30,login,<NA>,2020-12-07,EU,Android,19:35:55
423757,9385A108F5A0A7A7,2020-12-30,login,<NA>,2020-12-07,EU,PC,10:54:15
423758,DB650B7559AC6EAC,2020-12-30,login,<NA>,2020-12-07,EU,Android,10:59:09
423759,F80C9BDDEA02E53C,2020-12-30,login,<NA>,2020-12-07,EU,iPhone,09:53:39


In [32]:
def add_mkt_event(df_users, df_mkt_events): 
    """ 
    Adds the name of the marketing event from df_mkt_events to df_users if the event_dt is within start_dt and finish_dt and the region matches. 
    Parameters: 
    - df_users: DataFrame with columns ['user_id', 'event_dt', 'region', ...] 
    - df_mkt_events: DataFrame with columns ['name', 'regions', 'start_dt', 'finish_dt'] 
    
    Returns: 
    - df_users with new column 'mkt_event_name' 
    """ 
    
    # Function to apply per row of df_users 
    def find_event(row): 
        # Filter events where the user's date is within the range 
        mask_date = ((df_mkt_events['start_dt'] <= row['event_dt']) & (df_mkt_events['finish_dt'] >= row['event_dt'])) 
        
        # Filter by region using sets, row by row 
        def region_match(event_regions_str): 
            event_regions_set = set(event_regions_str.replace('_', ' ').split()) 
            user_regions_set = set(row['region'].split()) 
            return bool(event_regions_set & user_regions_set) # Non-empty intersection 
        
        mask_region = df_mkt_events['regions'].apply(region_match) 
        
        # Events that meet both conditions 
        matched_events = df_mkt_events[mask_date & mask_region] 
        
        if not matched_events.empty: 
            return matched_events.iloc[0]['name'] 
        else: return pd.NA 
    
    df_users['mkt_event_name'] = df_users.apply(find_event, axis=1) 
    return df_users

In [33]:
df_users = add_mkt_event(df_users, df_mkt_events)
df_users

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name
0,E1BDDCE0DAFA2679,2020-12-07,purchase,99.99,2020-12-07,N.America,iPhone,20:22:03,<NA>
1,7B6452F081F49504,2020-12-07,purchase,9.99,2020-12-07,EU,iPhone,09:22:53,<NA>
2,9CD9F34546DF254C,2020-12-07,purchase,4.99,2020-12-07,N.America,iPhone,12:59:29,<NA>
3,96F27A054B191457,2020-12-07,purchase,4.99,2020-12-07,EU,iPhone,04:02:40,<NA>
4,1FD7660FDF94CA1F,2020-12-07,purchase,4.99,2020-12-07,EU,Android,10:15:09,<NA>
...,...,...,...,...,...,...,...,...,...
423756,245E85F65C358E08,2020-12-30,login,<NA>,2020-12-07,EU,Android,19:35:55,Christmas_New_Year_Promo
423757,9385A108F5A0A7A7,2020-12-30,login,<NA>,2020-12-07,EU,PC,10:54:15,Christmas_New_Year_Promo
423758,DB650B7559AC6EAC,2020-12-30,login,<NA>,2020-12-07,EU,Android,10:59:09,Christmas_New_Year_Promo
423759,F80C9BDDEA02E53C,2020-12-30,login,<NA>,2020-12-07,EU,iPhone,09:53:39,Christmas_New_Year_Promo


In [34]:
df_users = df_users.merge(df_user_test, on='user_id', how='left')
df_users

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
0,E1BDDCE0DAFA2679,2020-12-07,purchase,99.99,2020-12-07,N.America,iPhone,20:22:03,<NA>,NaN,NaN
1,7B6452F081F49504,2020-12-07,purchase,9.99,2020-12-07,EU,iPhone,09:22:53,<NA>,NaN,NaN
2,9CD9F34546DF254C,2020-12-07,purchase,4.99,2020-12-07,N.America,iPhone,12:59:29,<NA>,NaN,NaN
3,96F27A054B191457,2020-12-07,purchase,4.99,2020-12-07,EU,iPhone,04:02:40,<NA>,B,interface_eu_test
4,1FD7660FDF94CA1F,2020-12-07,purchase,4.99,2020-12-07,EU,Android,10:15:09,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
429471,245E85F65C358E08,2020-12-30,login,<NA>,2020-12-07,EU,Android,19:35:55,Christmas_New_Year_Promo,NaN,NaN
429472,9385A108F5A0A7A7,2020-12-30,login,<NA>,2020-12-07,EU,PC,10:54:15,Christmas_New_Year_Promo,NaN,NaN
429473,DB650B7559AC6EAC,2020-12-30,login,<NA>,2020-12-07,EU,Android,10:59:09,Christmas_New_Year_Promo,NaN,NaN
429474,F80C9BDDEA02E53C,2020-12-30,login,<NA>,2020-12-07,EU,iPhone,09:53:39,Christmas_New_Year_Promo,A,interface_eu_test


### 📝 **6: Pre-test Funnel Analisis**

#### **6.1** Funnel data consistency

In [35]:
# Funnel stages should be: login, product_page, product_cart, purchase
df_users_irregular = detect_irregular_funnel_sequences(df_users)
print(f"Users registries with irregular sequence: {len(df_users_irregular)}")
df_users_irregular

Users registries with irregular sequence: 112834


,user_id,event_date,event_sequence
0,000199F1887AE5E6,2020-12-14,"[(2020-12-14 09:56:09, purchase), (2020-12-14 ..."
1,000199F1887AE5E6,2020-12-15,"[(2020-12-15 07:22:56, purchase), (2020-12-15 ..."
2,000199F1887AE5E6,2020-12-20,"[(2020-12-20 06:36:35, purchase), (2020-12-20 ..."
3,000199F1887AE5E6,2020-12-21,"[(2020-12-21 02:11:23, purchase), (2020-12-21 ..."
4,0002499E372175C7,2020-12-22,"[(2020-12-22 03:49:52, purchase), (2020-12-22 ..."
...,...,...,...
112829,FFF91B6C5431F375,2020-12-14,"[(2020-12-14 22:12:04, product_cart), (2020-12..."
112830,FFF91B6C5431F375,2020-12-17,"[(2020-12-17 08:27:17, product_cart), (2020-12..."
112831,FFFFE36C0F6E92DF,2020-12-22,"[(2020-12-22 11:38:57, product_cart), (2020-12..."
112832,FFFFE36C0F6E92DF,2020-12-23,"[(2020-12-23 05:09:13, product_page), (2020-12..."


In [36]:
# Filter irregular users' registries
df_users = df_users.merge(df_users_irregular[['user_id', 'event_date']], left_on=['user_id', 'event_dt'], right_on=['user_id', 'event_date'], how='left', indicator=True)
df_users = df_users.loc[(df_users['_merge'] == 'left_only'), :].drop(columns=['_merge', 'event_date'])
df_users

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
2,9CD9F34546DF254C,2020-12-07,purchase,4.99,2020-12-07,N.America,iPhone,12:59:29,<NA>,NaN,NaN
19,9C31B4124B3AE217,2020-12-07,purchase,4.99,2020-12-07,EU,PC,21:38:36,<NA>,NaN,NaN
21,FCD216B91578B8DC,2020-12-07,purchase,99.99,2020-12-07,N.America,PC,18:24:41,<NA>,NaN,NaN
28,AA77BDA8996FE1B8,2020-12-07,purchase,9.99,2020-12-07,N.America,Android,08:24:41,<NA>,NaN,NaN
36,2765321AC15BA00A,2020-12-07,purchase,4.99,2020-12-07,EU,Mac,08:57:39,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
429464,2761935C2DD2794F,2020-12-30,login,<NA>,2020-12-07,EU,iPhone,16:51:29,Christmas_New_Year_Promo,NaN,NaN
429465,E0E5446B78A6520B,2020-12-30,login,<NA>,2020-12-07,CIS,Android,23:36:30,CIS_New_Year_Gift_Lottery,NaN,NaN
429468,33E7BCF696B41C7B,2020-12-30,login,<NA>,2020-12-07,EU,PC,12:21:24,Christmas_New_Year_Promo,NaN,NaN
429472,9385A108F5A0A7A7,2020-12-30,login,<NA>,2020-12-07,EU,PC,10:54:15,Christmas_New_Year_Promo,NaN,NaN


#### **6.2** Funnel

In [37]:
# Get conversions rate
df_funnel = df_users.groupby('event_name', observed=True).agg(events=('event_name', 'count'), users=('user_id', 'nunique')).sort_values(by='users', ascending=False).reset_index()
df_funnel['conversion_rate'] = ((df_funnel['users'] / df_funnel['events']) * 100).round(3)
df_funnel['total_conversion_rate'] = ((df_funnel['users'] / df_funnel.loc[0, 'users']) * 100).round(3)
df_funnel['stage_conversion_rate'] = ((df_funnel['users'] / df_funnel['users'].shift(1)) * 100).round(3)
df_funnel

,event_name,events,users,conversion_rate,total_conversion_rate,stage_conversion_rate
0,login,70757,30763,43.477,100.000,NaN
1,product_page,33627,17650,52.488,57.374,57.374
2,product_cart,10907,6373,58.430,20.716,36.108
3,purchase,171,163,95.322,0.530,2.558


In [38]:
plot_horizontal_bar_plotpx(df_funnel, x='users', y='event_name', title='Pre-Test Conversion Funnel', xlabel='Users', ylabel='Events', sort=True)

In [39]:
plot_horizontal_bar_plotpx(df_funnel, x='stage_conversion_rate', y='event_name', title='Pre-Test Stage Conversion Rate Funnel', xlabel='Rate', ylabel='Events', sort=True)

#### 🧪 **7: A/B Testing**

__AB Test Technical Description__

- Test Name: __recommender_system_test__
- Groups: A (control), B (new checkout funnel)
- Launch Date: 2020-12-07
- Date new users stopped: 2020-12-21
- End Date: 2021-01-01
- Audience: __15% of new__ users from the _EU region_
- Test Purpose: To test changes related to the introduction of an improved recommendation system
- Expected Outcome: __Within 14 days of enrollment__, users will show improved conversion rates for product page views (the product_page event), adding items to the shopping cart (product_cart), and purchases (purchase). At each stage of the `product_page → product_cart → purchase` __funnel__, there will be _at least a 10% increase_.
- Expected number of test participants: 6,000

#### **7.1** Dataset test requirements fitting 

In [40]:
# Get data first date lifetime
display(HTML(f"> Earliest first date in df_users: <b>{df_users['first_date'].min()}</b>"))
display(HTML(f"> Latest first date in df_users: <b>{df_users['first_date'].max()}</b>"))

In [41]:
# Get data that fits onñy for A/B test lifetime (Launch - End date)
df_users_testAB = df_users.loc[(df_users['first_date'] >= dt.date(2020, 12, 7)) & (df_users['first_date'] <= dt.date(2020, 12, 21)), :]
df_users_testAB

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
2,9CD9F34546DF254C,2020-12-07,purchase,4.99,2020-12-07,N.America,iPhone,12:59:29,<NA>,NaN,NaN
19,9C31B4124B3AE217,2020-12-07,purchase,4.99,2020-12-07,EU,PC,21:38:36,<NA>,NaN,NaN
21,FCD216B91578B8DC,2020-12-07,purchase,99.99,2020-12-07,N.America,PC,18:24:41,<NA>,NaN,NaN
28,AA77BDA8996FE1B8,2020-12-07,purchase,9.99,2020-12-07,N.America,Android,08:24:41,<NA>,NaN,NaN
36,2765321AC15BA00A,2020-12-07,purchase,4.99,2020-12-07,EU,Mac,08:57:39,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
429464,2761935C2DD2794F,2020-12-30,login,<NA>,2020-12-07,EU,iPhone,16:51:29,Christmas_New_Year_Promo,NaN,NaN
429465,E0E5446B78A6520B,2020-12-30,login,<NA>,2020-12-07,CIS,Android,23:36:30,CIS_New_Year_Gift_Lottery,NaN,NaN
429468,33E7BCF696B41C7B,2020-12-30,login,<NA>,2020-12-07,EU,PC,12:21:24,Christmas_New_Year_Promo,NaN,NaN
429472,9385A108F5A0A7A7,2020-12-30,login,<NA>,2020-12-07,EU,PC,10:54:15,Christmas_New_Year_Promo,NaN,NaN


In [42]:
# Get audience Info within Test AB

html_output =""

for region in df_users_testAB['region'].unique():
    if region == df_users_testAB['region'].unique()[-1]:
        html_output += f"{region}"
    else:
        html_output += f"{region}, "
    
display(HTML(f"> Regions within Test AB: <b>{html_output}</b>"))

In [43]:
# Double check whether the amount of audience is correct after filtering test AB data

html_output =""
audience_AB_Total = 0
audience_AB_NAmerica = 0
audience_AB_EU = 0
audience_AB_APAC = 0
audience_AB_CIS = 0

for region in df_users_testAB['region'].unique():
    if region == 'N.America':
        audience_AB_NAmerica = df_users_testAB.loc[(df_users_testAB['region'] == 'N.America'), 'user_id'].unique().shape[0]
    elif region == 'EU':
        audience_AB_EU = df_users_testAB.loc[(df_users_testAB['region'] == 'EU'), 'user_id'].unique().shape[0]
    elif region == 'APAC':
        audience_AB_APAC = df_users_testAB.loc[(df_users_testAB['region'] == 'APAC'), 'user_id'].unique().shape[0]
    else:
        audience_AB_CIS = df_users_testAB.loc[(df_users_testAB['region'] == 'CIS'), 'user_id'].unique().shape[0]

audience_AB_Total = audience_AB_APAC + audience_AB_CIS + audience_AB_EU + audience_AB_NAmerica

display(HTML(f"> Total audience in test AB (All regions): <b>{audience_AB_Total}</b>"))

for region in df_users_testAB['region'].unique():
    if region == 'N.America':
        display(HTML(f"> N.america audience rate in test AB: <b>{round((audience_AB_NAmerica / audience_AB_Total) * 100, 3)} %</b>"))
    elif region == 'EU':
        display(HTML(f"> EU audience rate in test AB: <b>{round((audience_AB_EU / audience_AB_Total) * 100, 3)} %</b>"))
    elif region == 'APAC':
        display(HTML(f"> APAC audience rate in test AB: <b>{round((audience_AB_APAC / audience_AB_Total) * 100, 3)} %</b>"))
    else:
        display(HTML(f"> CISC audience rate in test AB: <b>{round((audience_AB_CIS / audience_AB_Total) * 100, 3)} %</b>"))

In [44]:
gray_scale = ['#2b2b2b', '#4d4d4d', '#707070', '#999999', '#bfbfbf']
plot_pie_chart_plotlypx(df_users_testAB, 'region', 'user_id', color_map=gray_scale, labels=True, title="Audience proportions", unique_values=True)

#### **7.2** A/B groups segmentation

In [45]:
# Get data by groups
df_users_group_a = df_users_testAB.loc[(df_users['group'] == 'A'), :]
df_users_group_a

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
41,F2BE35774F63059B,2020-12-07,purchase,4.99,2020-12-07,EU,Android,05:23:18,<NA>,A,interface_eu_test
271,F80C9BDDEA02E53C,2020-12-07,purchase,4.99,2020-12-07,EU,iPhone,10:05:54,<NA>,A,interface_eu_test
523,F0715ADC532E14B4,2020-12-07,purchase,4.99,2020-12-07,EU,Android,01:57:58,<NA>,A,interface_eu_test
777,8FF91E21E27A330D,2020-12-07,purchase,4.99,2020-12-07,EU,Mac,01:57:30,<NA>,A,interface_eu_test
971,0BA0B65F1B8C19F9,2020-12-07,purchase,4.99,2020-12-07,EU,Mac,04:42:52,<NA>,A,interface_eu_test
...,...,...,...,...,...,...,...,...,...,...,...
429393,E5589EAE02ACD150,2020-12-29,login,<NA>,2020-12-20,EU,Mac,22:17:08,Christmas_New_Year_Promo,A,recommender_system_test
429396,D21F0D4FDCD82DB2,2020-12-29,login,<NA>,2020-12-20,EU,iPhone,02:17:00,Christmas_New_Year_Promo,A,recommender_system_test
429407,C15DC7DF26A3300D,2020-12-29,login,<NA>,2020-12-20,EU,Android,02:57:31,Christmas_New_Year_Promo,A,interface_eu_test
429411,BCEC881B3C573B2B,2020-12-29,login,<NA>,2020-12-20,EU,PC,08:17:16,Christmas_New_Year_Promo,A,interface_eu_test


In [46]:
df_users_group_b = df_users_testAB.loc[(df_users['group'] == 'B'), :]
df_users_group_b

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
88,392C03684E704CB3,2020-12-07,purchase,4.99,2020-12-07,EU,PC,09:45:50,<NA>,B,interface_eu_test
128,406CD606E407DF53,2020-12-07,purchase,4.99,2020-12-07,EU,iPhone,05:12:23,<NA>,B,interface_eu_test
134,A4145A2EA1E17654,2020-12-07,purchase,499.99,2020-12-07,EU,PC,16:15:16,<NA>,B,interface_eu_test
158,C7CB2F1BA42F102B,2020-12-07,purchase,99.99,2020-12-07,EU,iPhone,03:05:05,<NA>,B,recommender_system_test
682,3242BDDFA690A22B,2020-12-07,purchase,4.99,2020-12-07,EU,Android,18:10:54,<NA>,B,recommender_system_test
...,...,...,...,...,...,...,...,...,...,...,...
429369,98DEF9BB002BA9E3,2020-12-29,login,<NA>,2020-12-20,EU,iPhone,13:16:10,Christmas_New_Year_Promo,B,interface_eu_test
429378,8B02FD26DBC4FDE3,2020-12-29,login,<NA>,2020-12-20,EU,PC,10:29:02,Christmas_New_Year_Promo,B,interface_eu_test
429432,8CE3B6FD918462B4,2020-12-29,login,<NA>,2020-12-20,EU,PC,17:55:44,Christmas_New_Year_Promo,B,interface_eu_test
429438,2C29721DDDA76B2A,2020-12-29,login,<NA>,2020-12-20,EU,iPhone,05:58:20,Christmas_New_Year_Promo,B,interface_eu_test


#### **7.3** A/A Test
An AA test consists of randomly dividing users into two groups (A1 and A2), without applying any differentiation between them. Both groups receive the same experience, interface, recommendations, etc.   

AA Test Objective:   
- Randomly divide users into two groups: AA1 and AA2
- Do not apply any differences between them
- Verify that the funnel metrics (login → product_page → product_cart → purchase) are statistically equivalent
- Detect bias, noise, or assignment errors before introducing changes

**7.3.1** AA group segmentation

In [64]:
# Assign AA1 or AA2 group
df_users_aa_base = df_users_group_a.sort_values(by='user_id').reset_index(drop=True)
df_users_aa_base

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
0,0002CE61FF2C4011,2020-12-08,product_page,<NA>,2020-12-07,EU,Mac,22:14:19,<NA>,A,interface_eu_test
1,0002CE61FF2C4011,2020-12-08,product_cart,<NA>,2020-12-07,EU,Mac,22:14:20,<NA>,A,interface_eu_test
2,0002CE61FF2C4011,2020-12-08,login,<NA>,2020-12-07,EU,Mac,22:14:18,<NA>,A,interface_eu_test
3,003346BB64227D0C,2020-12-19,login,<NA>,2020-12-17,EU,Android,22:46:58,<NA>,A,interface_eu_test
4,003346BB64227D0C,2020-12-24,login,<NA>,2020-12-17,EU,Android,11:28:16,<NA>,A,interface_eu_test
...,...,...,...,...,...,...,...,...,...,...,...
14820,FFAE9489C76F352B,2020-12-13,product_page,<NA>,2020-12-13,EU,Mac,18:46:06,<NA>,A,recommender_system_test
14821,FFAE9489C76F352B,2020-12-15,login,<NA>,2020-12-13,EU,Mac,20:14:51,<NA>,A,recommender_system_test
14822,FFAE9489C76F352B,2020-12-19,product_page,<NA>,2020-12-13,EU,Mac,11:32:51,<NA>,A,recommender_system_test
14823,FFDC1BEFD27A66D5,2020-12-10,login,<NA>,2020-12-10,EU,iPhone,09:31:29,<NA>,A,interface_eu_test


In [67]:
df_users_aa_a1 = df_users_aa_base.iloc[0:7413, :]
df_users_aa_a1

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
0,0002CE61FF2C4011,2020-12-08,product_page,<NA>,2020-12-07,EU,Mac,22:14:19,<NA>,A,interface_eu_test
1,0002CE61FF2C4011,2020-12-08,product_cart,<NA>,2020-12-07,EU,Mac,22:14:20,<NA>,A,interface_eu_test
2,0002CE61FF2C4011,2020-12-08,login,<NA>,2020-12-07,EU,Mac,22:14:18,<NA>,A,interface_eu_test
3,003346BB64227D0C,2020-12-19,login,<NA>,2020-12-17,EU,Android,22:46:58,<NA>,A,interface_eu_test
4,003346BB64227D0C,2020-12-24,login,<NA>,2020-12-17,EU,Android,11:28:16,<NA>,A,interface_eu_test
...,...,...,...,...,...,...,...,...,...,...,...
7408,818112D4190F52DF,2020-12-10,product_page,<NA>,2020-12-07,EU,iPhone,07:25:16,<NA>,A,interface_eu_test
7409,818112D4190F52DF,2020-12-18,product_page,<NA>,2020-12-07,EU,iPhone,03:12:24,<NA>,A,interface_eu_test
7410,818112D4190F52DF,2020-12-10,login,<NA>,2020-12-07,EU,iPhone,07:25:15,<NA>,A,interface_eu_test
7411,81836000CB0E6C20,2020-12-07,login,<NA>,2020-12-07,EU,iPhone,13:08:36,<NA>,A,interface_eu_test


In [69]:
df_users_aa_a2 = df_users_aa_base.iloc[7413:, :]
df_users_aa_a2

,user_id,event_dt,event_name,details,first_date,region,device,event_tm,mkt_event_name,group,ab_test
7413,818AEF8DE8B21EA7,2020-12-14,login,<NA>,2020-12-14,EU,Android,11:50:05,<NA>,A,recommender_system_test
7414,818AEF8DE8B21EA7,2020-12-15,login,<NA>,2020-12-14,EU,Android,12:49:18,<NA>,A,recommender_system_test
7415,818AEF8DE8B21EA7,2020-12-27,login,<NA>,2020-12-14,EU,Android,09:53:59,Christmas_New_Year_Promo,A,recommender_system_test
7416,81AD59E23B9825CE,2020-12-20,product_cart,<NA>,2020-12-16,EU,Android,09:12:40,<NA>,A,recommender_system_test
7417,81AD59E23B9825CE,2020-12-20,product_page,<NA>,2020-12-16,EU,Android,09:12:39,<NA>,A,recommender_system_test
...,...,...,...,...,...,...,...,...,...,...,...
14820,FFAE9489C76F352B,2020-12-13,product_page,<NA>,2020-12-13,EU,Mac,18:46:06,<NA>,A,recommender_system_test
14821,FFAE9489C76F352B,2020-12-15,login,<NA>,2020-12-13,EU,Mac,20:14:51,<NA>,A,recommender_system_test
14822,FFAE9489C76F352B,2020-12-19,product_page,<NA>,2020-12-13,EU,Mac,11:32:51,<NA>,A,recommender_system_test
14823,FFDC1BEFD27A66D5,2020-12-10,login,<NA>,2020-12-10,EU,iPhone,09:31:29,<NA>,A,interface_eu_test


In [71]:
users_aa_a1 = df_users_aa_a1['user_id'].unique() 
users_aa_a1

array(['0002CE61FF2C4011', '003346BB64227D0C', '00341D8401F0F665', ...,
       '8180B27CEE324BFD', '818112D4190F52DF', '81836000CB0E6C20'],
      dtype=object)

In [72]:
users_aa_a2 = df_users_aa_a2['user_id'].unique() 
users_aa_a2

array(['818AEF8DE8B21EA7', '81AD59E23B9825CE', '81AE85FD2E4A0B25', ...,
       'FF825C1D791989B5', 'FFAE9489C76F352B', 'FFDC1BEFD27A66D5'],
      dtype=object)

In [ ]:
# Check if there are the same users in both samples
duplicates = set(df_users_aa_a1['user_id'].unique()) & set(df_users_aa_a2['user_id'].unique())
print(f"> Duplicate users between AA1 and AA2: {len(duplicates)}")


> Duplicate users between AA1 and AA2: 0


In [80]:
# Check group balance
total = df_users_aa_a1['user_id'].nunique() + df_users_aa_a2['user_id'].nunique()
difference = df_users_aa_a1['user_id'].nunique() - df_users_aa_a2['user_id'].nunique()
rate = (difference / total) * 100

display(HTML(f"> Group balance verification: <i>AA1 group</i> (<b>{df_users_aa_a1['user_id'].nunique()}</b>), <i>AA2 group</i> (<b>{df_users_aa_a2['user_id'].nunique()}</b>)"))
display(HTML(f"> Group balance difference rate: <i>Difference</i> <b>{difference}</b>, Rate <b>{round(rate, 3)} %</b>"))

if rate < 5:
    display(HTML(f"> Groups are balanced."))
else:
    display(HTML(f"> Groups are unbalanced."))

**7.3.2** AA groups conversion comparison

In [81]:
df_funnel_AA1 = df_users_aa_a1
df_funnel_AA1 = df_funnel_AA1.groupby('event_name', observed=True).agg(events=('event_name', 'count'), users=('user_id', 'nunique')).sort_values(by='users', ascending=False).reset_index()
df_funnel_AA1['conversion_rate'] = ((df_funnel_AA1['users'] / df_funnel_AA1['events']) * 100).round(3)
df_funnel_AA1['total_conversion_rate'] = ((df_funnel_AA1['users'] / df_funnel_AA1.loc[0, 'users']) * 100).round(3)
df_funnel_AA1['stage_conversion_rate'] = ((df_funnel_AA1['users'] / df_funnel_AA1['users'].shift(1)) * 100).round(3)
df_funnel_AA1

,event_name,events,users,conversion_rate,total_conversion_rate,stage_conversion_rate
0,login,4626,1947,42.088,100.000,NaN
1,product_page,2095,1086,51.838,55.778,55.778
2,product_cart,690,394,57.101,20.236,36.280
3,purchase,2,2,100.000,0.103,0.508


In [82]:
df_funnel_AA2 = df_users_aa_a2
df_funnel_AA2 = df_funnel_AA2.groupby('event_name', observed=True).agg(events=('event_name', 'count'), users=('user_id', 'nunique')).sort_values(by='users', ascending=False).reset_index()
df_funnel_AA2['conversion_rate'] = ((df_funnel_AA2['users'] / df_funnel_AA2['events']) * 100).round(3)
df_funnel_AA2['total_conversion_rate'] = ((df_funnel_AA2['users'] / df_funnel_AA2.loc[0, 'users']) * 100).round(3)
df_funnel_AA2['stage_conversion_rate'] = ((df_funnel_AA2['users'] / df_funnel_AA2['users'].shift(1)) * 100).round(3)
df_funnel_AA2

,event_name,events,users,conversion_rate,total_conversion_rate,stage_conversion_rate
0,login,4574,1930,42.195,100.000,NaN
1,product_page,2137,1115,52.176,57.772,57.772
2,product_cart,687,378,55.022,19.585,33.901
3,purchase,14,13,92.857,0.674,3.439


In [83]:
# Z test

# 1. Hypotheses H₀, H₁
# H₀: The proportion of events are the same between group AA1 and group AA2.
# H₁: The proportion of events are different between group AA1 and group AA2.

# 2. Specify Significance or Confidence
# alpha = 5%
# confidence = 95%

alpha = 0.05

# 3. Calculate critical and test values, define acceptance and rejection zones

results = []

# Total number of users in each group (make sure it is not empty)
try:
    total_users_1 = int(df_funnel_AA1.loc[df_funnel_AA1['event_name'] == 'login', 'users'].values[0])
    total_users_2 = int(df_funnel_AA2.loc[df_funnel_AA2['event_name'] == 'login', 'users'].values[0])
except IndexError:
    raise ValueError("The 'login' event was not found in one of the groups.")

for event in df_funnel_AA1['event_name']:
    try:
        count1 = int(df_funnel_AA1.loc[df_funnel_AA1['event_name'] == event, 'users'].values[0])
        count2 = int(df_funnel_AA2.loc[df_funnel_AA2['event_name'] == event, 'users'].values[0])

        # Validation: avoid division by zero
        if total_users_1 == 0 or total_users_2 == 0:
            stat, pval = np.nan, np.nan
        else:
            stat, pval = proportions_ztest(count=[count1, count2], nobs=[total_users_1, total_users_2])

        results.append({
            'event': event,
            'AA1_users': count1,
            'AA2_users': count2,
            'z_score': round(stat, 4) if not np.isnan(stat) else None,
            'p_value': round(pval, 4) if not np.isnan(pval) else None
        })

    except IndexError:
        print(f"Event '{event}' not found in both groups. Ignored.")
    
df_ztest = pd.DataFrame(results)
df_ztest

c:\Users\luisp\anaconda3\envs\ai_dataAnalysis\lib\site-packages\statsmodels\stats\weightstats.py:792: RuntimeWarning:

invalid value encountered in scalar divide



,event,AA1_users,AA2_users,z_score,p_value
0,login,1947,1930,NaN,NaN
1,product_page,1086,1115,-1.2530,0.2102
2,product_cart,394,378,0.5073,0.6119
3,purchase,2,13,-2.8627,0.0042


In [84]:
for index, row in df_ztest.iterrows():
    if row['event'] == 'login':
        continue
    elif row['p_value'] < 0.05:
        display(HTML(f"> H₀ is rejected, not rejecting H₁, because there is enough statistical evidence that the proportion of <i>{row['event']}</i> events differs significantly between group AA1 and group AA2."))
    else:
        display(HTML(f"> H₀ is not rejected, rejecting H₁, because there is not enough statistical evidence that the proportion of <i>{row['event']}</i> events differs between group AA1 and group AA2."))


#### **7.4** A/B Test

**7.4.1** AA group segmentation